In [1]:
from sedona.spark import SedonaContext

config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# Here we define a variable with the URI to our data in the S3 bucket
geoparquet = 's3://wherobots-examples/data/onboarding_1/nyc_buildings.parquet'

# Then we can load that into a Sedona DataFrame 
buildings = sedona.read.format("geoparquet").load(geoparquet)

In [6]:
buildings.printSchema()

root
 |-- BUILD_ID: integer (nullable = true)
 |-- OCC_CLS: string (nullable = true)
 |-- PRIM_OCC: string (nullable = true)
 |-- SEC_OCC: string (nullable = true)
 |-- PROP_ADDR: string (nullable = true)
 |-- PROP_CITY: string (nullable = true)
 |-- PROP_ST: string (nullable = true)
 |-- PROP_ZIP: string (nullable = true)
 |-- OUTBLDG: string (nullable = true)
 |-- HEIGHT: float (nullable = true)
 |-- SQMETERS: float (nullable = true)
 |-- SQFEET: float (nullable = true)
 |-- H_ADJ_ELEV: float (nullable = true)
 |-- L_ADJ_ELEV: float (nullable = true)
 |-- FIPS: string (nullable = true)
 |-- CENSUSCODE: string (nullable = true)
 |-- PROD_DATE: timestamp (nullable = true)
 |-- SOURCE: string (nullable = true)
 |-- USNG: string (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- IMAGE_NAME: string (nullable = true)
 |-- IMAGE_DATE: timestamp (nullable = true)
 |-- VAL_METHOD: string (nullable = true)
 |-- REMARKS: string (nullable = tr

In [3]:
# Once again we can define our S3 URI to our Raster dataset
central_park = 's3://wherobots-examples/data/onboarding_1/CentralPark.tif'

# Here we can use the Sedona Spatial SQL functions to load the data in using the RS_FromPath function 
elevation =sedona.read\
                 .format("raster")\
                 .option("tileWidth", "256")\
                 .option("tileHeight", "256")\
                 .load(central_park)

In [4]:
elevation.createOrReplaceTempView('elevation')

In [5]:
elevation.show()

+--------------------+---+---+---------------+
|                rast|  x|  y|           name|
+--------------------+---+---+---------------+
|OutDbGridCoverage...|  1| 30|CentralPark.tif|
|OutDbGridCoverage...|  0| 27|CentralPark.tif|
|OutDbGridCoverage...| 26| 10|CentralPark.tif|
|OutDbGridCoverage...| 12| 17|CentralPark.tif|
|OutDbGridCoverage...|  8| 36|CentralPark.tif|
|OutDbGridCoverage...| 13| 40|CentralPark.tif|
|OutDbGridCoverage...|  5| 20|CentralPark.tif|
|OutDbGridCoverage...| 10|  6|CentralPark.tif|
|OutDbGridCoverage...| 35| 38|CentralPark.tif|
|OutDbGridCoverage...|  9| 48|CentralPark.tif|
|OutDbGridCoverage...|  5| 25|CentralPark.tif|
|OutDbGridCoverage...|  9| 14|CentralPark.tif|
|OutDbGridCoverage...| 10| 25|CentralPark.tif|
|OutDbGridCoverage...| 34| 26|CentralPark.tif|
|OutDbGridCoverage...| 25| 26|CentralPark.tif|
|OutDbGridCoverage...| 18| 46|CentralPark.tif|
|OutDbGridCoverage...|  7| 12|CentralPark.tif|
|OutDbGridCoverage...| 21| 50|CentralPark.tif|
|OutDbGridCov

Creation the point geometry. RS_Value function to get the raster value for that specific point location

In [8]:
specific_point_elevation = sedona.sql('''
SELECT
    RS_Value(rast, ST_Point(-73.9751781, 40.7756813)) as elevation_in_feet
FROM
    elevation 
''')

In [11]:
# filter using a where clause 
specific_point_elevation.where("elevation_in_feet is not NULL").show()

+-----------------+
|elevation_in_feet|
+-----------------+
|             90.0|
+-----------------+



In [12]:
import json
from sedona.maps.SedonaKepler import SedonaKepler

#  load  map configuration so it can be read by SedonaKepler. 
with open('map-config/config.json') as f:
    # Load the JSON data into a dictionary
    map_config = json.load(f)

# map with configuration and the dataframe. 
map = SedonaKepler.create_map(config=map_config)
SedonaKepler.add_df(map, buildings, 'NYC Buildings')
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 's8z1snp', 'type': …

Analyzing Building Elevations

Get the elevation avg of all the buildings 

In [13]:
buildings.createOrReplaceTempView('buildings')

In [14]:
sedona.sql('SELECT RS_SRID(rast) FROM elevation limit 1').show()

+-------------+
|rs_srid(rast)|
+-------------+
|         2263|
+-------------+



In [15]:
buildings_elevation = sedona.sql(f'''with a as (
SELECT
    buildings.PROP_ADDR as name,
    buildings.geom,
    avg(RS_ZonalStats(elevation.rast, st_transform(buildings.geom, 'epsg:4326','epsg:2263'), 1, 'mean', true)) as elevation
FROM
    buildings
JOIN
    elevation
ON
    RS_Intersects(elevation.rast, st_transform(buildings.geom, 'epsg:4326','epsg:2263'))
GROUP BY
    buildings.PROP_ADDR, buildings.geom)

SELECT * FROM a WHERE elevation > 0
''')

In [16]:
buildings_elevation.show()

+--------------------+--------------------+------------------+
|                name|                geom|         elevation|
+--------------------+--------------------+------------------+
|122 79 ST TRANSVERSE|MULTIPOLYGON (((-...|131.04895104895104|
|                NULL|MULTIPOLYGON (((-...| 57.17829037614608|
|        830 5 AVENUE|MULTIPOLYGON (((-...| 45.05213856813123|
|                NULL|MULTIPOLYGON (((-...|112.67299578059071|
|        830 5 AVENUE|MULTIPOLYGON (((-...|46.016948524321734|
|2 CENTRAL PK NEAR...|MULTIPOLYGON (((-...| 47.16783216783217|
|       1000 5 AVENUE|MULTIPOLYGON (((-...|105.91815235008104|
|151 97 ST TRANSVERSE|MULTIPOLYGON (((-...|  88.3225790000194|
|151 97 ST TRANSVERSE|MULTIPOLYGON (((-...| 91.44938560561734|
|        830 5 AVENUE|MULTIPOLYGON (((-...| 45.10060967912953|
|        830 5 AVENUE|MULTIPOLYGON (((-...| 45.02614968440036|
|    50 TERRACE DRIVE|MULTIPOLYGON (((-...| 81.48911564625851|
|      3 CENTER DRIVE|MULTIPOLYGON (((-...| 59.11810755

In [17]:
with open('map-config/central_park_config.json') as f:
    # Load the JSON data into a dictionary
    park_config = json.load(f)

map = SedonaKepler.create_map(config=park_config)
SedonaKepler.add_df(map, buildings_elevation, 'NYC Buildings')
map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'ouiidim', 'type': …